In [23]:
import numpy as np
import pandas as pd

from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

In [2]:
DATASET_PATH = "dataset/"
EDGES_PATH = f"{DATASET_PATH}edges/"
EMBEDDINGS_PATH = f"{DATASET_PATH}embeddings/"
TRAIN_TEST_GRAPHS_PATH = f"{DATASET_PATH}train_test_graphs/"
FOLLOW_NET_PATH = f"{EDGES_PATH}follow_edges.csv"
IGNORE_NET_PATH = f"{EDGES_PATH}ignore_edges.csv"
POS_VOTE_NET_PATH = f"{EDGES_PATH}pos_vote_edges.csv"
NEG_VOTE_NET_PATH = f"{EDGES_PATH}neg_vote_edges.csv"
REPLY_NET_PATH = f"{EDGES_PATH}reply_edges.csv"

In [3]:
follow_edges = pd.read_csv(FOLLOW_NET_PATH, sep=" ", header=None, names=["source", "target"])
ignore_edges = pd.read_csv(IGNORE_NET_PATH, sep=" ", header=None, names=["source", "target"])
pos_vote_edges = pd.read_csv(POS_VOTE_NET_PATH, sep=" ", header=None, names=["source", "target"])
neg_vote_edges = pd.read_csv(NEG_VOTE_NET_PATH, sep=" ", header=None, names=["source", "target"])
reply_edges = pd.read_csv(REPLY_NET_PATH, sep=" ", header=None, names=["source", "target", "weight"])

## Step 1 Prepare StellarGraph Object

In [4]:
# Create StellarGraph object from edge list data 
G = StellarGraph(edges=follow_edges, is_directed=True)
print(G.info())

# Constants
NETWORK_NAME = "follow"
SAMPLING_PERCENTAGE = 0.1
TRAIN_SIZE = 0.8
DEV_SIZE = 1-TRAIN_SIZE # Fixed

StellarGraph: Directed multigraph
 Nodes: 15530, Edges: 57117

 Node types:
  default: [15530]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [57117]
        Weights: all 1 (default)
        Features: none


In [5]:
import random

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

## Step 2 Define Test, Train & Dev Split

Just like with regular machine learning, we have to split the data, in order to correctly train and evaluate the results, without data leakage.
We need the following components:

- Train Graph: `G_train`; We compute node embeddings on this graph.
- Training Set: `examples_train` (labels are `labels_train`); A training set of positive and negative edges. These edges must not be used to compute node embeddings.
- Dev Set: `examples_dev` (labels are `labels_dev`); 
- Test Graph: `G_test`; We compute test node embeddings on this graph, which contains more edges than the train graph.
- Test Set: `examples_test` (labels are `labels_test`); A test set of positive and negative edges. These edges must not be used to compute test node embeddings and also not be part of the training or dev set.


In [6]:
# Define an edge splitter on the full graph for the test graph creation
edge_splitter_test = EdgeSplitter(G)

# Randomly select SAMPLING_PERCENTAGE of all positive links and same number of negative links
# from full graph and save the reduced graph as G_test (does not contain sampled links)
G_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=SAMPLING_PERCENTAGE,
    method="global",
    seed=SEED
)

print(G_test.info())
print(examples_test.shape)

** Sampled 5711 positive and 5711 negative edges. **
StellarDiGraph: Directed multigraph
 Nodes: 15530, Edges: 51406

 Node types:
  default: [15530]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [51406]
        Weights: all 1 (default)
        Features: none
(11422, 2)


In [7]:
# Define an edge splitter on the test graph for the train graph creation
edge_splitter_train = EdgeSplitter(G_test, G)

# Randomly select SAMPLING_PERCENTAGE of all positive links and same number of negative links
# from test graph reduced graph as graph_train
G_train, examples_train_dev, labels_train_dev = edge_splitter_train.train_test_split(
    p=SAMPLING_PERCENTAGE,
    method="global",
    seed=SEED
)

print(G_train.info())
print(examples_train_dev.shape)

** Sampled 5140 positive and 5140 negative edges. **
StellarDiGraph: Directed multigraph
 Nodes: 15530, Edges: 46266

 Node types:
  default: [15530]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [46266]
        Weights: all 1 (default)
        Features: none
(10280, 2)


In [8]:
# Actual train-test-split with sklearn
(
    examples_train,
    examples_dev,
    labels_train,
    labels_dev,
) = train_test_split(
    examples_train_dev,
    labels_train_dev,
    train_size=TRAIN_SIZE,
    test_size=DEV_SIZE,
    random_state=SEED
)

overview_train_textsplit = pd.DataFrame(
    [
        (
            "Train set",
            len(examples_train),
            "Test Graph",
            "Train link classifier",
        ),
        (
            "Dev set",
            len(examples_dev),
            "Test Graph",
            "Select best link classfier",
        ),
        (
            "Test set",
            len(examples_test),
            "Full Graph",
            "Evaluate best link classier",
        )
    ],
    columns=("Type", "Examples", "Picked from", "Purpose"),
)

overview_train_textsplit.set_index("Type", inplace=True)
overview_train_textsplit

,Examples,Picked from,Purpose
Type,,,
Train set,8224,Test Graph,Train link classifier
Dev set,2056,Test Graph,Select best link classfier
Test set,11422,Full Graph,Evaluate best link classier


Save the train and test graph as an edgelist, such that OpenNE can use it as input to produce embeddings.

In [9]:
G_follow_train_df = pd.DataFrame(G_train.edges())
G_follow_train_df.to_csv(f"{TRAIN_TEST_GRAPHS_PATH}{NETWORK_NAME}_train.csv", sep=" ", header=False, index=False)

G_follow_test_df = pd.DataFrame(G_test.edges())
G_follow_test_df.to_csv(f"{TRAIN_TEST_GRAPHS_PATH}{NETWORK_NAME}_test.csv", sep=" ", header=False, index=False)

## Step 3 Embeddings

### 3.1 Embeddings with OpenNE

The following cell constructs the OpenNE commands that have to be run, in order to produce the embeddings needed in the next step.

In [103]:
EMBEDDING_METHOD = "sdne"
PARAMS = {
    "encoder-list": [8192, 128],
    "alpha": 1e-6,
    "beta": 10,
    "bs": 16,
    "nu1": 1e-3,
    "nu2": 1e-2,
    "lr": 0.000001
}

In [104]:
with open(f"{EMBEDDINGS_PATH}{EMBEDDING_METHOD}{'_params2' if PARAMS else ''}.sh", "w") as f:
    for mode in ["train", "test"]:
        openne_cmd = f"""python -m openne --method {EMBEDDING_METHOD}
        --directed --graph-format edgelist
        --input {TRAIN_TEST_GRAPHS_PATH}{NETWORK_NAME}_{mode}.csv
        --output {EMBEDDINGS_PATH}{NETWORK_NAME}_{mode}_{EMBEDDING_METHOD}{'_params2' if PARAMS else ''}.txt
        {" ".join(f"--{key} {value}" for key, value in PARAMS.items())}"""
        openne_cmd_stripped = openne_cmd.replace("\n    ", " ")
        print(openne_cmd_stripped, file=f)

In [109]:
# This class wraps a given embedding, such that the embedding vector of any node can
# easily be looked up
class Embedding():
    def __init__(self, embedding_file):
        self.embedding_file = embedding_file
        self.embedding_df = pd.read_csv(embedding_file, sep=" ", header=None, skiprows=[0])

    def get_embedding(self, node):
        row = self.embedding_df[self.embedding_df[0] == node]
        row = np.reshape(row.to_numpy(), -1)[1:]
        if len(row) == 0:
            row = np.random.uniform(low=-1, high=1, size=(128,))
        return row

    def __str__(self):
        return self.embedding_file

embedding_train = Embedding(f"{EMBEDDINGS_PATH}{NETWORK_NAME}_train_{EMBEDDING_METHOD}{'_params' if PARAMS else ''}.txt")
embedding_test = Embedding(f"{EMBEDDINGS_PATH}{NETWORK_NAME}_test_{EMBEDDING_METHOD}{'_params' if PARAMS else ''}.txt")
print(embedding_train)
print(embedding_test)

dataset/embeddings/follow_train_sdne_params2.txt
dataset/embeddings/follow_test_sdne_params2.txt


### 3.2 GCN

In [ ]:
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding
from stellargraph.utils import plot_history
from tensorflow import keras

epochs = 30
train_generator = FullBatchLinkGenerator(G_train, method="gcn")
train_flow = train_generator.flow(examples_train_dev, labels_train_dev)
test_generator = FullBatchLinkGenerator(G_test, method="gcn")
test_flow = test_generator.flow(examples_test, labels_test)

gcn = GCN(
    layer_sizes=[16, 16],
    activations=["relu", "relu"],
    generator=train_generator,
    dropout=0.3
)

x_input, x_output = gcn.in_out_tensors()
prediction = LinkEmbedding(activation="relu", method="ip")(x_output)
prediction = keras.layers.Reshape((-1,))(prediction)

model = keras.Model(inputs=x_input, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=0.01),
    loss=keras.losses.binary_crossentropy,
    metrics=["AUC"],
)

init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print(f"\t{name}: {val:0.4f}")

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print(f"\t{name}: {val:0.4f}")

history = model.fit(
    train_flow, epochs=epochs, validation_data=test_flow, verbose=2, shuffle=False
)

plot_history(history)

## Step 4 Link Prediction

In [110]:
# This method calculates link embeddings, by applying a binary operator
# to the embeddings of the source and destination nodes of the link
# These link embeddings will be the features for link prediction
def link_examples_to_features(link_examples, embedding, binary_operator):
    features = []
    for src, dst in link_examples:
        features.append(
            binary_operator(
                embedding.get_embedding(src),
                embedding.get_embedding(dst)
            )
        )
    return features

# This method trains the link classifier
def train_link_prediction_model(link_examples, link_labels, embedding, binary_operator):
    clf = get_link_prediction_classifier()
    link_features = link_examples_to_features(
        link_examples, embedding, binary_operator
    )
    clf.fit(link_features, link_labels)
    return clf

# This method defines the classifier pipeline
def get_link_prediction_classifier(max_iter=2000):
    lr_clf = LogisticRegressionCV(
        cv=5,
        scoring="roc_auc",
        max_iter=max_iter,
        random_state=SEED
    )
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])

In [111]:
# This method evaluates the classifier
def evaluate_link_prediction_model(clf, link_examples_test, link_labels_test, embedding, binary_operator):
    link_features_test = link_examples_to_features(
        link_examples_test, embedding, binary_operator
    )
    score = evaluate_roc_auc(clf, link_features_test, link_labels_test)
    return score

# This method computes the ROC AUC score for predictions
def evaluate_roc_auc(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)

    # Check which class corresponds to positive links
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])

In [112]:
# Binary operators for the combination of the embeddings of linked nodes
def operator_hadamard(u, v):
    return u * v

def operator_l1(u, v):
    return np.abs(u - v)

def operator_l2(u, v):
    return (u - v) ** 2

def operator_avg(u, v):
    return (u + v) / 2.0

def run_link_prediction(binary_operator, embedding):
    clf = train_link_prediction_model(
        examples_train, labels_train, embedding, binary_operator
    )
    score = evaluate_link_prediction_model(
        clf,
        examples_dev,
        labels_dev,
        embedding,
        binary_operator,
    )

    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }

In [113]:
binary_operators = [operator_hadamard, operator_l1, operator_l2, operator_avg]

results = []
for b_op in binary_operators:
    res = run_link_prediction(b_op, embedding_train)
    results.append(res)

best_result = max(results, key=lambda result: result["score"])

print(f"Best result from '{best_result['binary_operator'].__name__}'")

pd.DataFrame(
    [(result["binary_operator"].__name__, result["score"]) for result in results],
    columns=("binary operator", "ROC AUC score"),
).set_index("binary operator")

Best result from 'operator_avg'


,ROC AUC score
binary operator,
operator_hadamard,0.814409
operator_l1,0.751871
operator_l2,0.747025
operator_avg,0.816009


In [114]:
best_result

{'classifier': Pipeline(steps=[('sc', StandardScaler()),
                 ('clf',
                  LogisticRegressionCV(cv=5, max_iter=2000, random_state=42,
                                       scoring='roc_auc'))]),
 'binary_operator': <function __main__.operator_avg(u, v)>,
 'score': 0.8160085098087092}

In [115]:
test_score = evaluate_link_prediction_model(
    best_result["classifier"],
    examples_test,
    labels_test,
    embedding_test,
    best_result["binary_operator"]
)
print(round(test_score, 6))

0.827033


## Results

In [116]:
results_df = pd.read_csv("link_prediction_results.txt", sep=" ")
results_df

,embedding,operator,dev,test
0,node2vec_params,operator_avg,0.813609,0.645681
1,node2vec,operator_avg,0.805272,0.437873
2,deepWalk_params,operator_avg,0.782664,0.486813
3,deepWalk,operator_avg,0.799085,0.432583
4,line_params,operator_hadamard,0.779505,0.735184
5,line,operator_avg,0.794700,0.565123
6,hope,operator_avg,0.829788,0.457719
7,sdne_params,operator_avg,0.828398,0.833068
8,sdne,operator_avg,0.812131,0.691118
9,sdne_params2,operator_avg,0.816009,0.827033
